In [1]:
import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from   config            import setup_seed
from   UKDALE_Parser     import UK_Dale_Parser
from   Electricity_model import ELECTRICITY
from NILM_Dataloader import NILMDataloader
from NILM_Dataset import NILMDataset
from   Trainer           import Trainer
import pickle            as pkl
import plotly.express as px
import plotly.io as pio
import pandas as pd
from pyprojroot import here

pio.renderers.default = 'iframe' # or 'colab' or 'iframe' or 'iframe_connected' or 'sphinx_gallery'

In [2]:
with open(here("results_UK-DALE_TitanV_kettle/uk_dale/kettle/results.pkl"), "rb") as f:
    res = pkl.load(f)

args = res["args"]

In [3]:
# override computing device
args.device = "cpu"
args.pretrain_num_epochs = 0
args.num_epochs = 0
args.validation_size = 1
args.hidden = 256

In [4]:
setup_seed(args.seed)

In [5]:
args.house_indices = [2]
args.validation_size = .1
ds_parser = UK_Dale_Parser(args)

In [6]:
model = ELECTRICITY(args)
trainer = Trainer(args,ds_parser,model)

In [7]:
# Why is this necessary?
model.pretrain = False

In [8]:
dataloader = NILMDataloader(args, ds_parser)
_, test_loader = dataloader.get_dataloaders()
mre, mae, acc, prec, recall, f1 = trainer.test(test_loader, map_location='cpu')

Test, rel_err 0.01, acc 0.99, f1 0.14: 100%|███████████████████████████| 7/7 [00:05<00:00,  1.17it/s]


In [9]:
print('Mean Accuracy:', acc)
print('Mean F1-Score:', f1)
print('MAE:', mae)
print('MRE:', mre)

Mean Accuracy: [0.99487537]
Mean F1-Score: [0.77862595]
MAE: [0.00756694]
MRE: [21.89523618]


In [10]:
inf_data_start = 0
inf_data_end = 480
inference_data = NILMDataset(
    x=ds_parser.x[-100000:],
    y=ds_parser.y[-100000:],
    status=ds_parser.status[-100000:],
    window_size=480,
    stride=480
)

In [11]:
inference_dataloader = torch.utils.data.DataLoader(
    dataset=inference_data,
    batch_size=inf_data_end-inf_data_start,
)

In [12]:
out = model.eval()

In [13]:
%%time
batches = []
for batch in inference_dataloader:
    batches.append(model(batch[0].to("cpu")))

CPU times: user 11.9 s, sys: 11 s, total: 22.9 s
Wall time: 3.06 s


In [14]:
%%time
data = []
for batch in batches:
    for chunk in batch[0]:
        data += list(chunk[0].tolist())

CPU times: user 2.35 ms, sys: 296 µs, total: 2.65 ms
Wall time: 8.66 ms


In [15]:
df_inf = pd.DataFrame(dict(
    #index = range(len(data)),
    #inf = data
    index = range(10000),
    inf = data[-10000:]
))

fig = px.line(df_inf, x="index", y="inf", line_shape="linear", render_mode="svg")
fig.show()

In [16]:
with open(here("data/UK_Dale/house_2/channel_8.dat")) as f:
    ground_truth = pd.read_csv(f, names=["time", "data"], sep=" ")

In [17]:
df_gt = pd.DataFrame(dict(
    index = range(10000),
    gt = ground_truth["data"][-10000:]
))

fig = px.line(df_gt, x="index", y="gt", line_shape="linear", render_mode="svg")
fig.show()

In [18]:
df_x = pd.DataFrame(dict(
    index = range(10000),
    x = ds_parser.x[-10000:]
))

fig = px.line(df_x, x="index", y="x", line_shape="linear", render_mode="svg")
fig.show()

In [19]:
ds_parser.x

array([-0.10393056, -0.10393056, -0.10393056, ..., -0.39086878,
       -0.39086878, -0.38874331])

In [20]:
ds_parser.y

array([0., 0., 0., ..., 0., 0., 0.])

In [21]:
px.line(df_x, x=range(len(ds_parser.x)), y=ds_parser.x, line_shape="linear", render_mode="svg")

In [22]:
px.line(df_x, x=range(len(ds_parser.y)), y=ds_parser.y, line_shape="linear", render_mode="svg")